In [30]:
from sklearn.manifold import MDS
import sys
import ipywidgets

from mathchords.io import ExperimentHandler
from mathchords.functions.characteristics import process

import mathchords.functions.gen_chords as gen
import mathchords.functions.clustering as cg
import mathchords.functions.characteristics as ce
import numpy as np
import mathchords.constans
from mathchords.constans import SCALES

In [31]:
reference_points=gen.Simula_inversiones(mathchords.constans.DO_basic_chords)
len(reference_points)

9

In [32]:
def separate_chords(chords):
    triads = []
    tetrads = []
    
    for chord in chords:
        if len(chord['intervals']) == 2:  # Las triadas tienen 2 intervalos (3 notas en total)
            triads.append(chord)
        elif len(chord['intervals']) == 3:  # Los acordes de 4 notas tienen 3 intervalos (4 notas en total)
            tetrads.append(chord)
    
    return triads, tetrads

#triads, tetrads=separate_chords(mathchords.constans.DO_chords)
triads, tetrads=separate_chords(reference_points)



In [33]:
BASE_PATH = "../data/"
handler = ExperimentHandler(BASE_PATH)
datos=ce.create_experiment_dict(triads)
#datos=ce.create_experiment_dict(tetrads)

chords=datos['chords']
results_structure=ce.process( datos, ce.transpose_to_zero)['results']
results_intervals_theory=ce.process(datos, ce.interval_vector)['results']
results_intervals_histo_12=ce.process(datos, ce.interval_histogram_with_dissmeasure)['results']
results_binary_notes=ce.process(datos, ce.binary_pitch_class_set)['results']
results_binary_intervals_12=ce.process(datos, ce.interval_histogram)['results']
results_polar_notes=ce.process(datos, ce.polar_pitch_classes)['results']
#results_polar_degree=ce.polar_degree(datos)
#print(results_intervals_theory.keys())
chord_ids = list(results_intervals_theory.keys())
def perform_mds(data, n_components=2):
    mds = MDS(n_components=n_components,  max_iter=30000,eps=1e-16)
    val=mds.fit_transform(data)
    print(mds.n_iter_,mds.stress_)
    return val,mds.dissimilarity_matrix_

261.6255653005986 329.6275569128699 0.3703907255508495
261.6255653005986 391.99543598174927 0.0521266964234618
329.6275569128699 391.99543598174927 0.4741198056768632
329.6275569128699 391.99543598174927 0.4741198056768632
329.6275569128699 523.2511306011972 0.00878685307938039
391.99543598174927 523.2511306011972 0.07798994489402729
391.99543598174927 523.2511306011972 0.07798994489402729
391.99543598174927 659.2551138257398 0.0012172647761089065
523.2511306011972 659.2551138257398 0.09767088023539988
261.6255653005986 311.1269837220809 0.6003738720888554
261.6255653005986 391.99543598174927 0.0521266964234618
311.1269837220809 391.99543598174927 0.28035027673272617
311.1269837220809 391.99543598174927 0.28035027673272617
311.1269837220809 523.2511306011972 0.004352817647162897
391.99543598174927 523.2511306011972 0.07798994489402729
391.99543598174927 523.2511306011972 0.07798994489402729
391.99543598174927 622.2539674441618 0.0038284265538862687
523.2511306011972 622.2539674441618 0

In [34]:
vector_encoding = [
    #results_structure[chord_id]['feature_vector'] 
    #+
      results_intervals_theory[chord_id]['feature_vector']
    for chord_id in results_intervals_theory.keys()
]

print(vector_encoding)
# Realiza MDS en los datos
vector_encoding_mds,matrix =ce.perform_mds2(vector_encoding)
cg.plot_chords_with_selection(vector_encoding_mds,matrix, chord_ids, results_intervals_theory,True )



[[0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 1, 0], [0, 0, 2, 0, 0, 1], [0, 0, 2, 0, 0, 1], [0, 0, 2, 0, 0, 1]]


Número de iteraciones: 50, Stress: 0.0
['C', 'C/E', 'C/G', 'Cm', 'Cm/D#', 'Cm/G', 'Cdim', 'Cdim/D#', 'Cdim/F#']


Dropdown(description='Chord ID:', options=(('Chord ID: chord_0, C', 'chord_0'), ('Chord ID: chord_1, C/E', 'ch…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': [Root: C<br>Bass: C<br>Octave: 4<br>Degree:
                            1<br>Intervals: [4, 3], Root: C<br>Bass: E<br>Octave:
                            4<br>Degree: 1<br>Intervals: [4, 3], Root: C<br>Bass:
                            G<br>Octave: 4<br>Degree: 1<br>Intervals: [4, 3], Root:
                            C<br>Bass: C<br>Octave: 4<br>Degree: 1<br>Intervals:
                            [3, 4], Root: C<br>Bass: D#<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 4], Root: C<br>Bass: G<br>Octave:
                            4<br>Degree: 1<br>Intervals: [3, 4], Root: C<br>Bass:
                            C<br>Octave: 4<br>Degree: 1<br>Intervals: [3, 3], Root:
                            C<br>Bass: D#<br>Octave: 4<br>Degree: 1<br>Intervals:
                            [3, 3], Root: C<br>Bass: F#<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 3]],
       

In [40]:
vector_encoding5 = [

     [10*x for x in results_intervals_histo_12[chord_id]['feature_vector']]
     #results_polar_degree[chord_id]['feature_vector'] +
     #results_intervals_histo_12[chord_id]['feature_vector']
     +
     results_binary_intervals_12[chord_id]['feature_vector']
     +
     results_binary_notes[chord_id]['feature_vector']
    #results_polar_notes[chord_id]['feature_vector']
    for chord_id in results_intervals_theory.keys()
]
i=0

for v in vector_encoding5:
    print(chord_ids[i],v)
    i+=1
# Realiza MDS en los datos
vector_encoding_mds5,matrix = ce.perform_mds2(vector_encoding5,'euclidean')
#vector_encoding_mds5_our, matrix2=ce.perform_mds2(vector_encoding5,'euclidean')
cg.plot_chords_with_selection(vector_encoding_mds5,matrix, chord_ids, results_binary_intervals_12,True)

chord_0 [0, 0, 4.741198056768631, 3.7039072555084953, 0, 0, 0.521266964234618, 0, 0, 0, 0, 8.966372276511745, 0.8151247524101586, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
chord_1 [0, 0, 4.741198056768631, 0, 0.7798994489402729, 0, 0, 0.0878685307938039, 0, 0, 0, 5.608966036502709, 0.5099060033184282, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
chord_2 [0, 0, 0, 0.9767088023539988, 0.7798994489402729, 0, 0, 0, 0.012172647761089065, 0, 0, 1.7687808990553604, 0.16079826355048732, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
chord_3 [0, 0, 6.003738720888554, 2.8035027673272617, 0, 0, 0.521266964234618, 0, 0, 0, 0, 9.328508452450434, 0.8480462229500395, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
chord_4 [0, 0, 0, 2.8035027673272617, 0.7798994489402729, 0, 0, 0, 0.043528176471628974, 0, 0, 3.626930392739163, 0.3297209447944694, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 

Dropdown(description='Chord ID:', options=(('Chord ID: chord_0, C', 'chord_0'), ('Chord ID: chord_1, C/E', 'ch…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': [Root: C<br>Bass: C<br>Octave: 4<br>Degree:
                            1<br>Intervals: [4, 3], Root: C<br>Bass: E<br>Octave:
                            4<br>Degree: 1<br>Intervals: [4, 3], Root: C<br>Bass:
                            G<br>Octave: 4<br>Degree: 1<br>Intervals: [4, 3], Root:
                            C<br>Bass: C<br>Octave: 4<br>Degree: 1<br>Intervals:
                            [3, 4], Root: C<br>Bass: D#<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 4], Root: C<br>Bass: G<br>Octave:
                            4<br>Degree: 1<br>Intervals: [3, 4], Root: C<br>Bass:
                            C<br>Octave: 4<br>Degree: 1<br>Intervals: [3, 3], Root:
                            C<br>Bass: D#<br>Octave: 4<br>Degree: 1<br>Intervals:
                            [3, 3], Root: C<br>Bass: F#<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 3]],
       

In [36]:

def inversion_to_frequencies(chord):
    """Calcula la inversión del acorde y convierte a frecuencias considerando octavas correctas."""
    A4_FREQ = 440  # Frecuencia de A4
    A4_NOTE = 9    # Valor numérico de A en el sistema de 12 semitonos
    A4_OCTAVE = 4  # Octava de A4

    reordered_sequence=ce.inversion_from_bass(chord)

    # Inicia la octava para la primera nota basada en el bajo y ajusta según la secuencia
    octave = chord['octave']
    previous_note = chord['bass']  # Comienza con el bajo como referencia
    frequencies = []

    for note in reordered_sequence:
        if note < previous_note:  # Si la nota actual es menor que la anterior, aumenta la octava
            octave += 1
        frequency = A4_FREQ * 2 ** (((note + (octave - A4_OCTAVE) * 12) - A4_NOTE) / 12)
        frequencies.append(frequency)
        previous_note = note  # Actualiza la nota anterior para la siguiente iteración

    return frequencies

def dissmeasure_fixed_amp(fvec, fixed_amp=1, model='min'):
    sort_idx = np.argsort(fvec)
    fr_sorted = np.asarray(fvec)[sort_idx]

    Dstar = 0.24
    S1 = 0.0207
    S2 = 18.96

    C1 = 5
    C2 = -5

    A1 = -3.51
    A2 = -5.75

    idx = np.transpose(np.triu_indices(len(fr_sorted), 1))
    fr_pairs = fr_sorted[idx]

    Fmin = fr_pairs[:, 0]
    S = Dstar / (S1 * Fmin + S2)
    Fdif = fr_pairs[:, 1] - fr_pairs[:, 0]

    # Usando una amplitud fija en lugar de variables basadas en el modelo
    a = fixed_amp  # Amplitud constante para todos los pares
    SFdif = S * Fdif
    D = np.sum(a * (C1 * np.exp(A1 * SFdif) + C2 * np.exp(A2 * SFdif)))

    return D

In [37]:
chords[6]

{'name': 'C Dim',
 'octave': 4,
 'bass': 0,
 'root': 0,
 'degree': 1,
 'intervals': [3, 3]}

In [38]:
ce.interval_histogram_with_dissmeasure(chords[6],6)

261.6255653005986 311.1269837220809 0.6003738720888554
261.6255653005986 369.9944227116344 0.10733635879192839
311.1269837220809 369.9944227116344 0.5055475368563775


{'chord': {'name': 'C Dim',
  'octave': 4,
  'bass': 0,
  'root': 0,
  'degree': 1,
  'intervals': [3, 3]},
 'feature_vector': [0,
  0,
  1.105921408945233,
  0,
  0,
  0.10733635879192839,
  0,
  0,
  0,
  0,
  0,
  1.2132577677371614,
  0.11029616070337832],
 'chord_id': 6}

In [39]:
ce.interval_histogram_with_dissmeasure(chords[3],3)

261.6255653005986 311.1269837220809 0.6003738720888554
261.6255653005986 391.99543598174927 0.0521266964234618
311.1269837220809 391.99543598174927 0.28035027673272617


{'chord': {'name': 'C Minor',
  'octave': 4,
  'bass': 0,
  'root': 0,
  'degree': 1,
  'intervals': [3, 4]},
 'feature_vector': [0,
  0,
  0.6003738720888554,
  0.28035027673272617,
  0,
  0,
  0.0521266964234618,
  0,
  0,
  0,
  0,
  0.9328508452450434,
  0.08480462229500395],
 'chord_id': 3}